In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
device = "cuda"

In [2]:
import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score

In [3]:
def evaluate(model, X, Y, params = ["fmeasure"]):
    torch.cuda.empty_cache()
    results = [] 
    predicted = []
    batch_size=1

    
    for i in range(int((len(X)/batch_size))):
        s = i*batch_size
        e = i*batch_size+batch_size
            
        inputs = Variable(X[s:e])
        pred = model(inputs)
            
        predicted.append(pred.data.cpu().numpy())
            
    predicted = torch.Tensor(predicted).view(-1, 1)

    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted).cpu())
            recall = recall_score(Y, np.round(predicted).cpu())
            results.append(2*precision*recall/ (precision+recall))
    return results

In [4]:
class LSTM(nn.Module):
    def __init__(self, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.lstm_input_size = 300
        self.lstm = nn.LSTM(self.lstm_input_size, hidden_size, num_layers, bidirectional=False, batch_first=True)
        self.linear = nn.Linear(128*200, self.num_classes)
        self.conv = nn.Conv2d(1, 128, (1, 18))
        self.batchnorm = nn.BatchNorm2d(128)



    def forward(self, x):

        out = self.conv(x)
        out = self.batchnorm(out)
        out = torch.squeeze(out)

        out, (h, c) = self.lstm(out)

        out = out.reshape(-1, 128*200)


        out = self.linear(out)

        #out = torch.squeeze(out)

        return out

In [5]:
model = LSTM(hidden_size= 200, num_layers = 1, num_classes = 1).cuda()

pred = model(torch.randn(120, 1, 300, 18).cuda())


#np.shape(pred)
#print(f"pred: {pred}")
#print(f"h: {h}")
#print(f"c: {c}")
print(np.shape(pred))
#summary(model.cuda(), (1, 300, 18))

torch.Size([120, 1])


In [6]:
X_train = torch.randn([1421, 1, 300, 18])
Y_train = torch.ones([1421, 1])
X_val = torch.randn([158, 1, 300, 18])
Y_val = torch.ones([158, 1])
X_test = torch.randn([1024, 1, 300, 18])
Y_test = torch.ones([1024, 1])

In [7]:
batch_size = 160
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)
torch.cuda.empty_cache()

In [8]:
for epoch in range(300):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    running_loss = 0.0
    for i in range(int(len(X_train)/batch_size-1)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(X_train[s:e].cuda())
        labels = Variable(Y_train[s:e].cuda())



        optimizer.zero_grad()
        outputs = model.forward(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        #print(loss.data)
    


Epoch  0

Epoch  1

Epoch  2

Epoch  3

Epoch  4

Epoch  5

Epoch  6

Epoch  7

Epoch  8

Epoch  9

Epoch  10

Epoch  11

Epoch  12

Epoch  13

Epoch  14

Epoch  15

Epoch  16

Epoch  17

Epoch  18

Epoch  19

Epoch  20

Epoch  21

Epoch  22

Epoch  23

Epoch  24

Epoch  25

Epoch  26

Epoch  27

Epoch  28

Epoch  29

Epoch  30

Epoch  31

Epoch  32

Epoch  33

Epoch  34

Epoch  35

Epoch  36

Epoch  37

Epoch  38

Epoch  39

Epoch  40

Epoch  41

Epoch  42

Epoch  43

Epoch  44

Epoch  45

Epoch  46

Epoch  47

Epoch  48

Epoch  49

Epoch  50

Epoch  51

Epoch  52

Epoch  53

Epoch  54

Epoch  55

Epoch  56

Epoch  57

Epoch  58

Epoch  59

Epoch  60

Epoch  61

Epoch  62

Epoch  63

Epoch  64

Epoch  65

Epoch  66

Epoch  67

Epoch  68

Epoch  69

Epoch  70

Epoch  71

Epoch  72

Epoch  73

Epoch  74

Epoch  75

Epoch  76

Epoch  77

Epoch  78

Epoch  79

Epoch  80

Epoch  81

Epoch  82

Epoch  83

Epoch  84

Epoch  85

Epoch  86

Epoch  87

Epoch  88

Epoch  89

Epoch  90

Epoch  9

In [19]:
torch.cuda.empty_cache()
results = [] 
predicted = []
batch_size=1
X = X_val.cuda()
Y = Y_val.cpu()
    
for i in range(int((len(X)/batch_size))):
    s = i*batch_size
    e = i*batch_size+batch_size
            
    inputs = Variable(X[s:e])
    pred = model(inputs)
            
    predicted.append(pred.data.cpu().numpy())
            
predicted = torch.Tensor(predicted).view(-1, 1)


accuracy_score(Y, np.round(predicted))


0.0